# Project resolution of optical dataset onto microwave dataset

**ToDo**

- shift pixels/ coordinates (+- 500 to each value)
- check several pixels if corners match
- need to figure out if bounds are universal for all pictures or have to bet individually which might be a paaain

- dataset.transform is the transformation function
- get pixels with transform function
- use Vedran's code
- 

In [1]:
import matplotlib.pyplot as plt
import numpy as np

import rioxarray
import xarray
import rasterio


%matplotlib inline

In [2]:
mw_path = r"../Data/microwave-rs/CumJour-North-ssmi-1979-2022-H19.nc"
opt_path = r"../Data/optical-rs/2019-06-08_grain_diameter.tif"

In [155]:
dataset_mw = xarray.open_dataset(mw_path).rio.write_crs(
    "epsg:3413", inplace=True
)  # add EPSG
dataset_mw = dataset_mw.where(
    dataset_mw["time"] == np.datetime64("2019-06-08"), drop=True
)  # only use one day for now
data_mw = dataset_mw["melt"]
data_opt = xarray.open_dataarray(opt_path)

In [4]:
data_mw.rio.to_raster("data_mw.tif")
data_omw_tif = xarray.open_dataarray("data_mw.tif")
data_omw_tif

<xarray.DataArray 'band_data' (band: 1, y: 109, x: 60)>
array([[[ 0.,  0., ..., nan, nan],
        [ 0.,  0., ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]]], dtype=float32)
Coordinates:
  * band         (band) int32 1
  * x            (x) float64 -6.5e+05 -6.25e+05 -6e+05 ... 8e+05 8.25e+05
  * y            (y) float64 -6.25e+05 -6.5e+05 ... -3.3e+06 -3.325e+06
    spatial_ref  int32 ...
Attributes:
    AREA_OR_POINT:  Area
    long_name:      Surface melt

In [5]:
def print_raster(raster):
    print(
        f"shape: \n   Y: {raster.rio.shape[0]} \n   X: {raster.rio.shape[1]}\n"
        f"resolution: {raster.rio.resolution()}\n"
        f"bounds: \n   minx = {raster.rio.bounds()[0]} \n   miny = {raster.rio.bounds()[1]} \n   maxx = {raster.rio.bounds()[2]} \n   maxy = {raster.rio.bounds()[3]}\n"
        f"sum: {raster.sum().item()}\n"
        f"CRS: {raster.rio.crs}\n"
    )


print("Microwave TIF Raster:\n----------------")
print_raster(data_omw_tif)
print("Optical Raster:\n----------------")
print_raster(data_opt)

Microwave TIF Raster:
----------------
shape: 
   Y: 109 
   X: 60
resolution: (25000.0, -25000.0)
bounds: 
   minx = -662500.0 
   miny = -3337500.0 
   maxx = 837500.0 
   maxy = -612500.0
sum: 358.0
CRS: EPSG:3413

Optical Raster:
----------------
shape: 
   Y: 2687 
   X: 1487
resolution: (1000.0, -1000.0)
bounds: 
   minx = -637000.0 
   miny = -3349000.0 
   maxx = 850000.0 
   maxy = -662000.0
sum: 324417.8125
CRS: EPSG:3413



In [9]:
with rasterio.open("data_mw.tif") as src_mw:
    with rasterio.open(opt_path) as src_opt:
        opt_tif = src_opt.read(1)
        mw_tif = src_mw.read(1)
        upscaled_mw = np.zeros(opt_tif.shape)
        for row, col in np.argwhere(~np.isnan(mw_tif)):
            lon, lat = rasterio.transform.xy(
                src_mw.transform, row, col, offset="center"
            )
            row_temp, col_temp = rasterio.transform.rowcol(src_opt.transform, lon, lat)
            upscaled_mw[row_temp, col_temp] = 1

# Test

In [92]:
from rasterio.enums import Resampling

upscale_factor = 25
new_width = data_mw.rio.width * upscale_factor
new_height = data_mw.rio.height * upscale_factor

data_mw_upsampled = data_mw.rio.reproject(
    data_mw.rio.crs,
    shape=(new_height, new_width),
    resampling=Resampling.bilinear,
)

del data_mw_upsampled.attrs["grid_mapping"]
data_mw_upsampled.rio.to_raster("data_mw_upsampled.tif")

c:\Users\ninaz\anaconda3\envs\my-env\lib\site-packages\rioxarray\raster_writer.py:115: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(


In [175]:
with rasterio.open("data_mw_upsampled.tif") as src_mw_upsampled:
    with rasterio.open("data_mw.tif") as src_mw:
        with rasterio.open(opt_path) as src_opt:
            upscaled_mw = np.zeros(src_mw_upsampled.shape)
            data_mw_full = xarray.open_dataset(mw_path).rio.write_crs(
                "epsg:3413", inplace=True
            )  # add EPSG
            for row, col in np.argwhere(~np.isnan(src_mw.read(1))):
                lon, lat = rasterio.transform.xy(
                    src_mw.transform, row, col, offset="center"
                )
                # bound= rasterio.transform.array_bounds(row, col,src_mw.transform)
                print(lon, lat)

                # row_new, col_new = rasterio.transform.rowcol(src_mw_upsampled.transform,lon,lat)
                # print(row, col, lon, lat)
            #    print(row, col, lon, lat)

-650000.0 -625000.0
-625000.0 -625000.0
-600000.0 -625000.0
-575000.0 -625000.0
-550000.0 -625000.0
-450000.0 -625000.0
-650000.0 -650000.0
-625000.0 -650000.0
-600000.0 -650000.0
-575000.0 -650000.0
-550000.0 -650000.0
-500000.0 -650000.0
-475000.0 -650000.0
-450000.0 -650000.0
-425000.0 -650000.0
-400000.0 -650000.0
-650000.0 -675000.0
-625000.0 -675000.0
-600000.0 -675000.0
-575000.0 -675000.0
-550000.0 -675000.0
-525000.0 -675000.0
-500000.0 -675000.0
-475000.0 -675000.0
-450000.0 -675000.0
-425000.0 -675000.0
-400000.0 -675000.0
-375000.0 -675000.0
-350000.0 -675000.0
-325000.0 -675000.0
125000.0 -675000.0
150000.0 -675000.0
175000.0 -675000.0
200000.0 -675000.0
-650000.0 -700000.0
-625000.0 -700000.0
-600000.0 -700000.0
-575000.0 -700000.0
-550000.0 -700000.0
-525000.0 -700000.0
-500000.0 -700000.0
-475000.0 -700000.0
-450000.0 -700000.0
-425000.0 -700000.0
-400000.0 -700000.0
-375000.0 -700000.0
-350000.0 -700000.0
-325000.0 -700000.0
75000.0 -700000.0
100000.0 -700000.0
125000.

In [ ]:
# VEDRANS CODE
"""
# manually upscale facebook data to same resolution
upscaled_fb = np.zeros(fb_resampled.shape)
for row, col in np.argwhere(fb > 0):
    # find lat long of the original point in the original raster
    lon, lat = rasterio.transform.xy(raster_trans,row,col,offset='center')
    # find the row col of the GPS point in the new reference system
    row, col = rasterio.transform.rowcol(resample_transform,lon,lat)
    # fill in the cell on the new raster
    #upscaled_fb[row,col] = 1
"""